In [25]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np

In [26]:
image_snake = Image.open("snake_bb_240.jpg")          # 뱀 사진을 open함수로 받아서 변수에 대입

In [27]:

class Joystick:
    def __init__(self):
        # 초기화 핀 설정
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        # SPI Baud Rate 설정
        self.BAUDRATE = 24000000
        # SPT 통신 초기화
        self.spi = board.SPI()
        # ST7789 Display Driver 초기화
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # 입력핀을 변수에 대입하여 버튼으로 사용:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # 백라이트 변수 설정
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # 조이스틱 폭과 높이 설정
        self.width = self.disp.width
        self.height = self.disp.height

In [28]:
class Snake:
    def __init__(self, width, height, edge):
        self.edge = edge                    # 뱀의 한 변의 길이 
        self.appearance = 'rectangle'       # 뱀의 형태
        self.state = 'alive'                # 뱀의 상태
        self.position = np.array([width/2 - edge, height/2 - edge, width/2 + edge, height/2 + edge])                    # 뱀의 4 좌표의 위치
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])  # 뱀의 중앙 위치
        self.outline = "#00FF00"  # 뱀의 테두리 색상
        self.img = self.fn_scale_snake(2*edge, self.center[0], self.center[1], width, height)  # 조정된 새로운 뱀의 이미지를 저장
    
    def fn_scale_snake(self, imsz, xc, yc, width, height) :     # imsz: 입력 이미지의 크기, xc: 뱀의 중심 x 좌표, yc: 뱀의 중심 y 좌표,
        x = imsz // 2 - xc                                      # width: 원하는 잘라낸 이미지의 너비, height: 원하는 잘라낸 이미지의 높이
        y = imsz // 2 - yc
        im = image_snake.resize((imsz, imsz), Image.BICUBIC)   # 입력 이미지의 크기를 지정된 크기 imsz로 조정
        im = im.crop((x, y, x + width, y + height))            # 조정된 이미지를 자름
        return im                                              # 이미지를 반환
        
    def move(self, command = None):
        
        # 뱀이 움직이지 않았을 때
        if command['move'] == False:
            self.state = None
            self.outline = "#FFFFFF"
        # 뱀이 움직였을 때 
        else:
            self.state = 'move'
            self.outline = "#FF0000" 

            # 위로 움직였을 때
            if command['up_pressed']:
                self.position[1] -= 5
                self.position[3] -= 5
                self.center[0] = (self.position[1] + self.position[3])/2

            # 아래로 움직였을 때
            if command['down_pressed']:
                self.position[1] += 5
                self.position[3] += 5
                self.center[0] = (self.position[1] + self.position[3])/2

            # 왼쪽으로 움직였을 때
            if command['left_pressed']:
                self.position[0] -= 5
                self.position[2] -= 5
                self.center[1] = (self.position[0] + self.position[2])/2
            
            # 오른쪽으로 움직였을 때
            if command['right_pressed']:
                self.position[0] += 5
                self.position[2] += 5
                self.center[1] = (self.position[0] + self.position[2])/2
    
    # 화면 경계와 충돌했는지 확인
    def check_collision_with_border(self, joystick):
        if self.center[0] < 0 or self.center[0] >= joystick.width:
            return True
        elif self.center[1] < 0 or self.center[1] >= joystick.height:
            return True
        
        return False

    # 뱀과 먹이가 충돌했는지 확인
    def check_collision_with_food(self, food):
        collision = self.overlap(self.position, food.position)           
        if collision:
            self.state = 'die'

    # 두 객체의 위치가 겹치는지 확인            
    def overlap(self, snake_position, food_position):
        return snake_position[0] < food_position[0] and snake_position[1] < food_position[1] and snake_position[2] > food_position[2] and snake_position[3] > food_position[3]

In [29]:
class Food:
    def __init__(self, spawn_position):
        self.appearance = 'circle'  #먹이의 형태
        self.state = 'alive'   #먹이의 상태
        self.position = np.array([spawn_position[0] - 5, spawn_position[1] - 5, spawn_position[0] + 5, spawn_position[1] + 5])  # 먹이의 4 좌표의 위치
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])    # 먹이의 중앙 위치
        self.outline = "#00FF00"         # 먹이의 테두리 색상

In [30]:
snake_size = [ 12    , 16       , 20     , 24     , 28      , 32      ]    # 뱀의 한 변의 길이를 배열로 구현
food_loc   = [(10,10), (150,150), (30,30), (90,90), (20,180), (180,20)]    # 먹이의 위치를 배열로 구현

# 폰트 크기 설정
fnt       = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 60) 
fnt_small = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 50)

joystick = Joystick()

# 게임을 초기화하도록 설정
def fn_initialize() :
    idx_snake = 0
    snake = Snake(joystick.width, joystick.height, snake_size[idx_snake])           # 뱀 객체 생성
    food  = Food(food_loc[idx_snake])                                               # 먹이 객체 생성
    my_draw =  ImageDraw.Draw(snake.img)                                            # snake.img를 Draw함수로 입력받아 draw 변수에 저장
    my_draw.rectangle(tuple(snake.position), outline = snake.outline)               # 뱀 형태를 이미지화
    my_draw.ellipse(tuple(food.position), outline = food.outline, fill = (255, 0, 0)) #먹이 형태를 이미지화
    return idx_snake, snake, food, my_draw

In [31]:
idx_snake, snake, food, my_draw = fn_initialize()
joystick.disp.image(snake.img)
start_time = time.time()

In [32]:
tf_run = True
while True:
    command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    
    if tf_run and not joystick.button_U.value:  # 위로 눌렀을 떄 위로 이동
        command['up_pressed'] = True
        command['move'] = True

    if tf_run and not joystick.button_D.value:  # 아래로 눌렀을 때 아래로 이동
        command['down_pressed'] = True
        command['move'] = True

    if tf_run and not joystick.button_L.value:  # 왼쪽으로 눌렀을 때 왼쪽으로 이동
        command['left_pressed'] = True
        command['move'] = True

    if tf_run and not joystick.button_R.value:  # 오른쪽으로 눌렀을 때 오른쪽으로 이동
        command['right_pressed'] = True
        command['move'] = True

    if not joystick.button_A.value:  # A 버튼을 눌렀을 때 게임 초기화
        idx_snake, snake, food, my_draw = fn_initialize()
        joystick.disp.image(snake.img)
        start_time = time.time()
        tf_run = True
        continue

    if not joystick.button_B.value:  # B 버튼을 눌렀을 때 Game Over 화면 출력
        snake.img = snake.fn_scale_snake(joystick.width, joystick.width//2, joystick.height//2, joystick.width, joystick.height)
        my_draw   = ImageDraw.Draw(snake.img)         
        my_draw.text((20,20), 'Game ',font=fnt, fill="#FF0000")
        my_draw.text((30,100), 'Over!',font=fnt, fill="#FF0000")
        joystick.disp.image(snake.img)
        break

    if not tf_run :
        continue

    snake.move(command)

    # 뱀이 화면 경계를 넘어갔을 때 뱀의 그림, 소요된 시간, "Out of Border" 텍스트를 출력
    if(snake.check_collision_with_border(joystick)):
        end_time = time.time()
        snake.img = snake.fn_scale_snake(joystick.width, joystick.width//2, joystick.height//2, joystick.width, joystick.height)
        my_draw   = ImageDraw.Draw(snake.img)
        my_draw.text((20,20),  'Out of', font=fnt, fill="#FF0000")
        my_draw.text((15,100), 'Border', font=fnt, fill="#FF0000")
        my_draw.text((20,180), '%.0f sec' % (end_time-start_time), font=fnt_small, fill="#FF0000")
        joystick.disp.image(snake.img)
        tf_run = False
        continue
    # 뱀의 먹이와 충돌했는지 확인
    snake.check_collision_with_food(food)
    
    # 뱀의 객체가 죽었는지 확인
    if( snake.state == 'die') :
        # 뱀의 객체가 남아있으면 다음 객체를 생성
        if( idx_snake+1 < len(snake_size) ) :
            idx_snake = idx_snake + 1
            snake = Snake(joystick.width, joystick.height, snake_size[idx_snake])
            food  = Food(food_loc[idx_snake])
        # 더 이상 존재하지 않는 경우 게임을 종료
        else:
            end_time = time.time()
            elapsed_time = end_time - start_time
            snake.img = snake.fn_scale_snake(joystick.width, joystick.width//2, joystick.height//2, joystick.width, joystick.height)
            my_draw   = ImageDraw.Draw(snake.img)         
            my_draw.text((20,20), 'Good ',font=fnt, fill="#FF7F00")
            my_draw.text((30,100), 'Job!',font=fnt, fill="#FF7F00")
            my_draw.text((20,180), '%.0f sec' % (end_time-start_time), font=fnt_small, fill="#FF7F00")
            joystick.disp.image(snake.img)
            tf_run = False             
            continue   
    # 뱀과 먹이를 이미지로 생성
    snake.img = snake.fn_scale_snake(2*snake.edge, snake.center[1], snake.center[0], joystick.width, joystick.height)
    my_draw =  ImageDraw.Draw(snake.img)
    my_draw.rectangle(tuple(snake.position), outline = snake.outline) 
    my_draw.ellipse(tuple(food.position), outline = food.outline, fill = (255, 0, 0))
    joystick.disp.image(snake.img)